In [1]:
# Need to define what a state looks like. I'm going to make the state a tuple that has both sides of the river represented
start_state = (("F","C","G","W"),())

In [2]:
# Just a quick helper function for printing
def state_to_string(s):
    return "".join(s[0])+"||"+"".join(s[1])

In [3]:
state_to_string(start_state)

'FCGW||'

In [4]:
# action(s) = possible actions
def remove_actions(letter,possible_actions,s,farmer_inx):
    if letter not in s[farmer_inx]:
        new_possible_actions = []
        for action in possible_actions:
            if letter not in action:
                new_possible_actions.append(action)
    else: # Nothing to do
        new_possible_actions = possible_actions
    return new_possible_actions

def action(s):
    # Find out which side the farmer is on
    farmer_inx = 0
    if 'F' in s[1]:
        farmer_inx = 1
    possible_actions = ["F>","FC>","FG>","FW>","F<","FC<","FG<","FW<"] # Same actions as above just a different order
    if farmer_inx == 0:
        possible_actions = possible_actions[0:4]
    else:
        possible_actions = possible_actions[4:8]
    
    possible_actions = remove_actions('C',possible_actions,s,farmer_inx)
    possible_actions = remove_actions('G',possible_actions,s,farmer_inx)
    possible_actions = remove_actions('W',possible_actions,s,farmer_inx)
    
    return possible_actions
    

In [5]:
action((('C', 'W'), ('F', 'G')))

['F<', 'FG<']

In [6]:
# succ(s,a) = apply action a while in state s, returns the new state
# we are going to assume only valid actions are requested
import copy
def succ(s,a):
    new_s = (list(s[0]),list(s[1]))
    #print(new_s)
    if '<' in a and "F" in new_s[1]: 
        # moving farmer from right to left
        new_s[1].remove("F")
        new_s[0].append("F")
        if len(a) == 3:
            new_s[1].remove(str(a[1]))
            new_s[0].append(str(a[1]))
    elif '>' in a and "F" in new_s[0]:
        new_s[0].remove("F")
        new_s[1].append("F")
        if len(a) == 3:
            new_s[0].remove(str(a[1]))
            new_s[1].append(str(a[1]))
    else:
        return None
    #print(new_s)
    return (tuple(new_s[0]),tuple(new_s[1]))

In [7]:
print(succ((('C', 'W'), ('F', 'G')),"F>"))

None


In [8]:
list(('F',))

['F']

In [9]:
# now code whether we are at the end
def isEnd(s):
    if len(s[0]) == 0:
        return True
    else:
        return False

In [10]:
isEnd((("W","F"),("C","G")))

False

In [11]:
isEnd(((),("W","F","C","G")))

True

In [12]:
# cost(s,a) = cost of action a in state s
def cost(current_s,a):
    s = succ(current_s,a)
    max_cost = 1000
    non_farmer_inx = 1
    if s is None:
        return max_cost
    # Find out which side the farmer is on
    if 'F' in s[1]:
        non_farmer_inx = 0

    # Need to check and see if goat and cabbage are there without farmer
    if "C" in s[non_farmer_inx] and "G" in s[non_farmer_inx]:
        return max_cost
    
    # now check wolf and goat
    if "W" in s[non_farmer_inx] and "G" in s[non_farmer_inx]:
        return max_cost
    
    if isEnd(s):
        return 0
    
    return 1

In [13]:
print(cost((("C","F","G"),("W")),"F>"))
print("\n")
print(cost((("C", "W"), ("F", "G")), "F>"))

1000


1000


# Finish the Code Below
Your task is to implement a breadth first search solution to the Goat, Sheep, Wolf Problem using the funtions above. Here is the function header and an example correct run.

In [68]:

# Now we are ready to implement a tree search
import numpy as np
from IPython.core.debugger import Tracer

def get_next_states(s, lst_a):
    next_states = []
    for x in lst_a:
        if succ(s, x) is not None:
            next_states.insert(0, (succ(s, x)))
        
    return set(next_states)

#I'm not sure why, but my method seems to get lost on a particular
# branch in the graph and then exit at the end of it, it's also not
# counting the right value, I know that. I think my inexperience with
# python has led to some bugs.
def bfs(start):
    visited, queue = set(), [start]
    count = 0
    while queue:
        vertex = queue.pop()
        count = count + 1
        next_steps = action(start)
        if vertex not in visited:
            visited.add(vertex)
            next_level = get_next_states(vertex, next_steps)
            for next in next_level:
                queue.append(next)
    return (count, visited)

In [69]:
bfs(start_state)

(5,
 {(('C', 'G'), ('F', 'W')),
  (('C', 'G', 'W'), ('F',)),
  (('C', 'W'), ('F', 'G')),
  (('F', 'C', 'G', 'W'), ()),
  (('G', 'W'), ('F', 'C'))})